In [ ]:
# import modules/libraries
import warnings 
warnings.simplefilter(action='ignore')
from entsoe import EntsoePandasClient
import pandas as pd
import json
import requests
from pandas import json_normalize
from bs4 import BeautifulSoup
from shapely.geometry import Polygon, LineString
import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import time
from datetime import datetime
from scipy import stats
from pathlib import Path
import itertools
import os
import pickle
import geojson
from sqlalchemy import create_engine
import re
import sqlite3
from pathlib import Path
from collections import OrderedDict

# geo libraries
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import chardet
from scipy import spatial
from scipy.spatial import KDTree
from shapely import wkt
from geopy.distance import geodesic
from geopandas import GeoDataFrame
import shapely.ops
import shapely.geometry
from shapely.geometry import Polygon, LineString
cwd = Path().resolve()

import sqlite3
# visualisation
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib as mpl 
%matplotlib inline 
import matplotlib.pyplot as plt
s = requests.Session()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#------
header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
   'Accept-Encoding': 'none',
   'Accept-Language': 'en-US,en;q=0.8',
   'Connection': 'keep-alive'}
##--

class InvalidCoordinate(Exception):
    pass

# conn = sqlite3.connect('source.db')
# vessel = pd.read_sql_query("SELECT imo FROM vessel_information", conn) #= pd.read_csv('./data/lng_tankers_lat.csv', index_col=False)

### 1. Scrape VesselFinder 

In [ ]:
"""
SQLITE: DELETE FROM lng WHERE date = '2022-12-28'

interesting news shipyard: https://www.ship-technology.com/news/daewoo-shipbuilding-contract-lng-ships/

"""
def get_current_vessel_location_V2(imo, delay=0.5):
    try:
        url = f"https://www.vesselfinder.com/en/vessels/VOS-TRAVELLER-IMO-{imo}"
        r = s.get(url, headers=header)
        r.raise_for_status()
        soup = BeautifulSoup(r.text)
        time.sleep(delay)
        
        data = json.loads(soup.find("div", {"id": "djson"})['data-json'])
        
        #  get latitude and longitude
        try:
            lat = data['ship_lat']
            lon = data['ship_lon']
        except:
            lat = np.nan
            lon = np.nan
    
        # get from and to port 
        try:
            _from = soup.find_all("a", {"class": "_npNa"})[0]['href']
            _from_text = soup.find_all("a", {"class": "_npNa"})[0].text
        except:
            _from = np.nan
            _from_text = np.nan
    
        try:
            _to = soup.find_all("a", {"class": "_npNa"})[1]['href']
            _to_text = soup.find_all("a", {"class": "_npNa"})[1].text
        except:
            _to = np.nan
            _to_text = np.nan

        try:
            ship_cog = data['ship_cog']
            ship_sog = data['ship_sog']
            ship_type = data['ship_type']
            no_pc = data['no_pc']
            mmsi = int(data['mmsi'])
        except:
            ship_cog = np.nan
            ship_sog = np.nan
            ship_type = np.nan
            no_pc = np.nan
            mmsi = np.nan
        
        # mining of all the dates
        today = datetime.now().strftime("%Y-%m-%d %H:%M")
        
        try:
            received = soup.find("td", {"class": "v3 ttt1 valm0"})['data-title']
            date_object = received.upper()
            date_object = datetime.strptime(date_object, "%b %d, %Y %H:%M UTC")
            date_object = date_object.strftime("%Y-%m-%d %H:%M")
            # print(date_object)
        except (TypeError, ValueError):
            date_object = np.nan
    
        try:
            eta_raw = soup.find("span", {"class": "_mcol12"}).text
            try:
                eta = eta_raw.split("ETA: ")[1].upper()
                # print(eta)
            except IndexError:
                eta = eta_raw.split("ATA: ")[1].upper()
            except:
                raise IndexError
        except AttributeError:
            # print(imo, "AtributesError", soup.find("span", {"class": "_mcol12"}))
            eta = np.nan
        except IndexError:
            eta = np.nan

        try:
            atd_raw = soup.find_all("div", {"class": "_value"})[1].text
        except (AttributeError, IndexError):
            atd_raw = np.nan

        # The latitude must be a number between -90 and 90 and the longitude between -180 and 180.
        if (lat < -90 or lat > 90) or (lon < -180 or lon > 180):
            raise InvalidCoordinate
        
        return pd.Series([lat, lon, _from, _to, date_object, eta, atd_raw, ship_cog, ship_sog, ship_type, mmsi, today])
    
    except InvalidCoordinate:
        print(f"Invalid Coordinate of vessel: {imo}")
        return pd.Series([lat, lon, _from, _to, date_object, eta, atd_raw, ship_cog, ship_sog, ship_type, mmsi, today])
    
    except Exception as e:
        input(f"Change vpn: {imo}")
        return pd.Series([lat, lon, _from, _to, date_object, eta, atd_raw, ship_cog, ship_sog, ship_type, mmsi, today])


def run():
    # print(f"start length: {vessel.shape[0]}")
    t0 = time.time()
    conn = sqlite3.connect('source.db')
    df = pd.read_sql_query("SELECT imo FROM vessel_information", conn)
    df.drop_duplicates(subset="imo", keep='first', inplace=True)
    df.dropna(subset=['imo'], inplace=True)
    # df = df.iloc[550:650,:]
    df2 = df.copy()
    df2[['lat', 'lon', 'fromPort', 'toPort', 'received', 'eta', 'atd',
         'ship_cog', 'ship_sog', 'ship_type', 'mmsi','scrape_date']] = df.apply(lambda x: get_current_vessel_location_V2(x['imo'], delay=0.1), axis=1)
    df2['mmsi'] = df2['mmsi'].fillna(0).astype('int64')
    print(f"Processing took: {round(time.time()-t0)} s") # ~6.4 min
    return df2

df = run()

#cwd = Path(__file__).parent
dirname = cwd.parent / "db"
db_name = 'file_both.db'
con = sqlite3.connect(db_name) #os.path.join(dirname, db_name)
cur = con.cursor()
create_table = (f'''CREATE TABLE IF NOT EXISTS VesselLocation (imo INT, lat TEXT, lon TEXT, fromPort TEXT, 
toPort TEXT, received TEXT, eta TEXT, atd TEXT, ship_cog TEXT, ship_sog TEXT, ship_type TEXT, no_pc TEXT,
mmsi, scrape_date TEXT)''')
cur.execute(create_table)

#create_table = (f'''CREATE TABLE IF NOT EXISTS VesselLocation (imo TEXT)''') #(f"""CREATE TABLE IF NOT EXISTS VesselLocation (rowid INTEGER PRIMARY KEY) WITHOUT ROWID;""")#



    
def write_to_db(con, cur, params):
    con.commit()
    
    """ dynamic addition of columns 
    columns = [i[1] for i in cur.execute('PRAGMA table_info(VesselLocation)')]
    for idx,i in enumerate(params['columns']):
        if i not in columns:
            #print(i)
            cur.execute(f'ALTER TABLE VesselLocation ADD COLUMN {i} TEXT')
            con.commit()
    """
    query_sign = ''.join(['?' if i == len(params['columns'])-1 else '?, ' for i in range(len(params['columns']))])
    cols = ''.join([i if i == params['columns'][-1] else i+', ' for i in params['columns']])
    sqlite_insert_with_param1 = f"""INSERT INTO VesselLocation  ({cols}) VALUES ({query_sign});"""
    data_tuple = tuple(params['values'])
    cur.execute(sqlite_insert_with_param1, data_tuple)
    con.commit()


title = df.columns.tolist()
for index, row in df.iterrows():
    results = OrderedDict()
    value = row.values.tolist()
    results['columns'] = title
    results['values'] = value
    write_to_db(con, cur, results)
    
con.close()


### 2. Scrape MarineTraffic

In [ ]:
session = requests.Session()
headers = {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate",
    "user-agent": "Mozilla/5.0",
    "x-requested-with": "XMLHttpRequest"
}
    
def get_ship_position(shipid, delay=0.5):
    """source: https://stackoverflow.com/questions/68654987/scraping-data-from-marinetraffic-page"""
    
    url = f"https://www.marinetraffic.com/vesselDetails/latestPosition/{shipid}"
    response = session.get(url, headers=headers)
    response.raise_for_status()
    time.sleep(delay)
    data = response.json()
    try:
        lat = data["lat"]
        lon = data["lon"]
    except:
        lat = np.nan
        lon = np.nan
    try:
        ts = datetime.utcfromtimestamp(data["lastPos"])
        date_object = ts.strftime("%Y-%m-%d %H:%M")
    except TypeError:
        ts = None
        date_object = np.nan
        
    # mining of all the dates
    today = datetime.now().strftime("%Y-%m-%d %H:%M")
    print("{}, {} Last known position: {} / {} @ {}".format(today, shipid, lat, lon, ts))
    return pd.Series([lat, lon, date_object, today])


def scrape_data():
    # print(f"start length: {vessel.shape[0]}")
    t0 = time.time()
    conn = sqlite3.connect('source.db')
    df = pd.read_sql_query("SELECT imo, shipid FROM marinetraffic_vessel_information", conn)
    df.drop_duplicates(subset="imo", keep='first', inplace=True)
    df.dropna(subset=['imo'], inplace=True)
    # df = df.iloc[70:100,:]
    df2 = df.copy()
    df2[['lat', 'lon', 'received','scrape_date']] = df.apply(lambda x: get_ship_position(x['shipid'], delay=0.1), axis=1)
    print(f"Processing took: {round(time.time()-t0)} s")
    return df2

df = scrape_data()
df.head()

def marinetraffic_write_to_db(con, cur, params):
    con.commit()
    query_sign = ''.join(['?' if i == len(params['columns'])-1 else '?, ' for i in range(len(params['columns']))])
    cols = ''.join([i if i == params['columns'][-1] else i+', ' for i in params['columns']])
    sqlite_insert_with_param1 = f"""INSERT INTO MarineTrafficVesselLocation ({cols}) VALUES ({query_sign});"""
    data_tuple = tuple(params['values'])
    cur.execute(sqlite_insert_with_param1, data_tuple)
    con.commit()

    
def saving(df):
    dirname = cwd.parent / "db"
    db_name = 'file_both.db'
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    create_table = (f'''CREATE TABLE IF NOT EXISTS MarineTrafficVesselLocation (imo INT, shipid TEXT, lat TEXT, lon TEXT,
    received TEXT, scrape_date DATE)''')
    cur.execute(create_table)

    title = df.columns.tolist()
    for index, row in df.iterrows():
        results = OrderedDict()
        value = row.values.tolist()
        results['columns'] = title
        results['values'] = value
        marinetraffic_write_to_db(con, cur, results)
    con.close()

saving(df)

### 3. Other Shit

    cur.execute("""CREATE VIEW [MarineTrafficVesselLocation-VIEW] AS SELECT imo, shipid, lat, lon, received, scrape_date
    FROM [MarineTrafficVesselLocation] order by rowid desc limit 1;""")

    cur.execute("""
    CREATE TRIGGER [MarineTrafficVesselLocation-TRIGGER] INSTEAD OF INSERT 
    ON [MarineTrafficVesselLocation-VIEW]

    WHEN new.received NOT IN (SELECT received FROM 
    [MarineTrafficVesselLocation-VIEW])

    BEGIN INSERT INTO [MarineTrafficVesselLocation] values(new.imo, new.shipid, new.lat, new.lon, new.received, new.scrape_date);
    END;""")

In [ ]:
def marinetraffic_write_to_db(con, cur, params):
    con.commit()
    query_sign = ''.join(['?' if i == len(params['columns'])-1 else '?, ' for i in range(len(params['columns']))])
    cols = ''.join([i if i == params['columns'][-1] else i+', ' for i in params['columns']])

    # del below:
#    sqlite_insert_with_param1 = 
#    f"""insert or replace into MarineTrafficVesselLocation (imo, shipid, lat, lon, received, scrape_date) values
#((select imo, received from MarineTrafficVesselLocation where received = "2022-12-28 09:40"), "2022-12-28 09:40", "2023-01-01 15:52");"""
#    f"""INSERT INTO MarineTrafficVesselLocation ({cols}) VALUES ({query_sign});"""

    #cur.execute("""INSERT INTO [MarineTrafficVesselLocation-VIEW] values(1113333,"shipid:4826396", 34.54853, 34.54853, "2022-12-28 09:40",
    #"2023-01-01 19:52");""")

    data_tuple = tuple(params['values'])
    print(data_tuple)
    cur.execute(f"""INSERT INTO [MarineTrafficVesselLocation-VIEW] ({cols}) VALUES ({query_sign});""", data_tuple)
    con.commit()

    
def saving(df):
    dirname = cwd.parent / "db"
    db_name = 'tmp_db4.sqlite'
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    create_table = (f'''CREATE TABLE IF NOT EXISTS MarineTrafficVesselLocation (imo INT, shipid TEXT, lat TEXT, lon TEXT,
    received DATE, scrape_date DATE)''')
    cur.execute(create_table)
    make = False
    # todo: set vor received instead of scrape_date 
    # change: order by scrape_date 
    if make:
        cur.execute("""CREATE VIEW [MarineTrafficVesselLocation-VIEW] AS SELECT DISTINCT imo, shipid, lat, lon, received, scrape_date
        FROM [MarineTrafficVesselLocation] order by scrape_date desc;""")

        cur.execute("""
        CREATE TRIGGER [MarineTrafficVesselLocation-TRIGGER] INSTEAD OF INSERT 
        ON [MarineTrafficVesselLocation-VIEW]

        WHEN new.scrape_date NOT IN (SELECT scrape_date FROM 
        [MarineTrafficVesselLocation-VIEW])

        BEGIN INSERT INTO [MarineTrafficVesselLocation] values(new.imo, new.shipid, new.lat, new.lon, new.received, new.scrape_date);
        END;""")
    

    title = df.columns.tolist()
    for index, row in df.iterrows():
        results = OrderedDict()
        value = row.values.tolist()
        results['columns'] = title
        results['values'] = value
        marinetraffic_write_to_db(con, cur, results)
    con.close()

# saving(df)

In [ ]:
def marinetraffic_write_to_db_V3(con, cur, params):
    con.commit()
    query_sign = ''.join(['?' if i == len(params['columns'])-1 else '?, ' for i in range(len(params['columns']))])
    cols = ''.join([i if i == params['columns'][-1] else i+', ' for i in params['columns']])


    data_tuple = tuple(params['values'])
    print(data_tuple)
    cur.execute(f"""INSERT INTO MarineTrafficVesselLocation (imo, shipid, lat, lon, received, scrape_date) 
    VALUES ({query_sign}) ON CONFLICT(imo, received) DO UPDATE SET imo={params['values'][0]}, shipid="{params['values'][1]}", 
    lat="{params['values'][2]}", lon="{params['values'][3]}", received="{params['values'][4]}",
    scrape_date="{params['values'][5]}";""", data_tuple)
    #DO UPDATE SET scrape_date=excluded.scrape_date;""", data_tuple)
    con.commit()

    
def saving(df):
    dirname = cwd.parent / "db"
    db_name = 'tmp_db8.sqlite'
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    create_table = (f'''CREATE TABLE IF NOT EXISTS MarineTrafficVesselLocation (imo INT UNIQUE, shipid TEXT, lat TEXT, lon TEXT,
    received DATE, scrape_date DATE)''')
    cur.execute(create_table)
    make = False
    # todo: set vor received instead of scrape_date 
    # change: order by scrape_date 
    if make:
        cur.execute("""CREATE VIEW [MarineTrafficVesselLocation-VIEW] AS SELECT DISTINCT imo, shipid, lat, lon, received, scrape_date
        FROM [MarineTrafficVesselLocation] order by scrape_date desc;""")

        cur.execute("""
        CREATE TRIGGER [MarineTrafficVesselLocation-TRIGGER] INSTEAD OF INSERT 
        ON [MarineTrafficVesselLocation-VIEW]

        WHEN new.scrape_date NOT IN (SELECT scrape_date FROM 
        [MarineTrafficVesselLocation-VIEW])

        BEGIN INSERT INTO [MarineTrafficVesselLocation] values(new.imo, new.shipid, new.lat, new.lon, new.received, new.scrape_date);
        END;""")
    

    title = df.columns.tolist()
    for index, row in df.iterrows():
        results = OrderedDict()
        value = row.values.tolist()
        results['columns'] = title
        results['values'] = value
        marinetraffic_write_to_db_V3(con, cur, results)
    con.close()

# saving(df)

In [ ]:
       """
    except requests.exceptions.Timeout:
        print("dfdf")
        # Maybe set up for a retry, or continue in a retry loop
    except requests.exceptions.TooManyRedirects:
        print("dddddddddd")
        # Tell the user their URL was bad and try a different one
    except requests.exceptions.RequestException as e:
        print("dfdsafdfadfadfadf")
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
        input("change vpn V2")
        return pd.Series([np.nan, np.nan, eta, atd_raw, ship_cog, ship_sog, ship_type, mmsi, today])
        """

In [ ]:
def test(name):
    # name_list = name.split(' ')
    # name_list = [i if len(i) >3 else None for i in name_list]
    # print(name_list)
    name = name.lower().replace('lng', '').replace(' al', '').replace('al ', '').replace('gas ', '').replace(' gas', '')
    for index, row in df_m3.iterrows():    
        #ship_name = row['Ship Name'].split(' ')
        #print(SequenceMatcher(a=name,b=row['Ship Name']).ratio())
        try:
            if SequenceMatcher(a=name,b=row['Ship Name'].lower()).ratio() > 0.4:
                print(name, row["Ship Name"].lower(), SequenceMatcher(a=name,b=row['Ship Name'].lower()).ratio())
                # return row["Ship Name"]
            #    return pd.Series([row["Ship Name"], row["Capacity (cu.m.)"]])
                return pd.Series([np.nan, np.nan])
            else:
                pass
        except AttributeError:
            # print(row['Ship Name'])
        except (TypeError, ValueError):
            pass
    return pd.Series([np.nan, np.nan])
        
vessel[['found','Capacity']] = vessel.apply(lambda x: test(x['ship']), axis=1)
vessel.head(100)

In [ ]:
import requests
from datetime import datetime
headers = {
        "accept": "application/json",
        "accept-encoding": "gzip, deflate",
        "user-agent": "Mozilla/5.0",
        "x-requested-with": "XMLHttpRequest"
}
    
def get_ship_position(url):
    """source: https://stackoverflow.com/questions/68654987/scraping-data-from-marinetraffic-page"""

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()


data = get_ship_position(f"https://www.marinetraffic.com/vesselDetails/latestPosition/shipid:{169195}")
ts = datetime.utcfromtimestamp(data["lastPos"])
print("Last known position: {} / {} @ {}".format(data["lat"], data["lon"], ts))


In [ ]:
"""
date_string = 'Dec 27, 2022 23:23 UTC'.upper()
date_object = datetime.strptime(date_string, "%b %d, %Y %H:%M UTC")
print("date_object =", date_object)
print("type of date_object =", type(date_object))



date_string = 'JAN 7, 14:00'.upper()
date_object = datetime.strptime(date_string, "%b %d, %H:%M")
print("date_object =", date_object)
print("type of date_object =", type(date_object))

"""

In [ ]:
conn = sqlite3.connect('source.db')
df = pd.read_sql_query("SELECT imo FROM vessel_information", conn) #= pd.read_csv('./data/lng_tankers_lat.csv', index_col=False)
df.dropna(subset=['imo'], inplace=True)
df.tail()

user_agent = {'User-agent': '14.0.3 Safari'}
session = requests.Session()
shipid = []
imo = []
vessel = []
mmsi = []
t0 = time.time()
for index, row in df.iterrows():
    url = f"https://www.marinetraffic.com/en/ais/details/ships/imo:{row['imo']}"
    r1 = session.get(url, headers=user_agent)
    url_split = r1.url.split('/')
    if len(url_split) == 11:
        shipid.append(url_split[-4])
        imo.append(row['imo'])
        mmsi.append(url_split[-3])
        vessel.append(url_split[-1])
        time.sleep(0.7)
        print(url_split)
    
df3 = pd.DataFrame(data={'imo': imo, 'shipid': shipid, 'mmsi': mmsi, 'vessel': vessel})
df3.head()
df3.to_sql('marinetraffic_vessel_information', conn, if_exists='replace', index = False)
print(f"Processing took: {round(time.time()-t0)} s")

In [ ]:
# list of fleet: https://www.nakilat.com/fleet-list/
conn = sqlite3.connect('source.db')
df_m3 = pd.read_html('http://www.aukevisser.nl/supertankers/gas-SP/id703.htm', skiprows = 0, header=0)[2]
df_m3.to_sql('aukevisser', conn, if_exists='replace', index = False)

In [ ]:
adsf